<img src="../common/rfsoc_book_banner.jpg" alt="University of Strathclyde" align="left">

# Notebook Set B

---

## 03 - ADCs and DACs
This notebook explores topics relating to the analogue-digital interface. These include aliasing and anti-aliasing filters for analogue to digital conversion and zero-order-hold techniques and reconstruction filters for digital to analogue conversion.

## Table of Contents
* [1. Introduction](#introduction)
* [2. Analogue to Digital Conversion](#adc)
    * [2.1. Aliasing](#aliasing)
    * [2.2. Anti-Aliasing Filter](#anti-aliasing-filter)
* [3. Digital to Analogue Conversion](#dac)
    * [3.1. Zero-Order-Hold](#zero-order-hold)
    * [3.2. Reconstruction Filter](#reconstruction-filter)
* [4. Upper Nyquist Zones](#upper-nyquist-zones)
* [5. Conclusion](#conclusion)

## Revision
* **v1.0** | 22/01/23 | *First Revision*

---


## 1. Introduction <a class="anchor" id="introduction"></a>
Before we begin exploring the analogue-digital conversion process, we will load a few Python libraries below.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rfsoc_book.helper_functions as hf
from scipy import signal

## 2. Analogue to Digital Conversion <a class="anchor" id="adc"></a>
We have already investigated two topics relating to analogue-digital conversion, which are sampling and quantisation. Aliasing is also of key importance when sampling an analogue signal, as higher frequency components (above the Nyquist sampling rate) can interfere with signals of interest.

An anti-aliasing filter is typically used to suppress the effects of aliasing, as shown in Figure 1. Depending on the sampling frequency of the ADC, it may be possible to use a combination of oversampling and a cheap analogue filter to acquire the frequency band of interest.

<figure>
<img src="./images/analogue_anti_alias.png" style="width: 75%;"/>
    <figcaption><b>Figure 1: Anti-aliasing filter placed before the Analogue to Digital Converter (ADC) to suppress the effects of aliasing. The top plot requires an expensive analogue filter to prevent aliasing. The bottom plot demonstrates the effects of oversampling the signal to use a cheaper analogue filter.</b></figcaption>
</figure>


### 2.1. Aliasing <a class="anchor" id="aliasing"></a>
We can easily demonstrate the effects of aliasing by sampling a signal that contains two frequency components. One frequency component will be within the Nyquist sampling rate and the other will be above the Nyquist sampling rate.

Consider the cell below, which approximates a continuous signal that contains a 50Hz and 240Hz sine wave and some Gaussian noise.

In [ ]:
Aa = 0.25   # Amplitude a
Ab = 0.5    # Amplitude b
fa = 50     # Desired frequency a
fb = 240    # Desired frequency b
t  = 0.2    # Time

fs_cont = 2400    # Sampling rate of 'continuous' signal

# Define 'continuous' signal for comparison
x_cont = np.arange(0, t, 1/fs_cont)
N_cont = np.size(x_cont)
noise  = np.random.normal(0, 0.01, N_cont)
y_cont = Aa * np.sin(2 * np.pi * fa * x_cont + 10) + Ab * np.sin(2 * np.pi * fb * x_cont) + noise

# Plot the 'continuous' signal
hf.plot_timeseries("Continuous Signal",
                   [x_cont], [y_cont], 
                   ['continuous'])

We can plot the log-scale power spectra of the 'continuous' signal to easily inspect its frequency content. Notice that there are two lobes, one at 50Hz and the other at 240Hz.

In [ ]:
# Compute the log-scale power spectra of the 'continuous' signal
Y_cont = np.fft.fft(y_cont)
Y_cont_norm = np.abs(Y_cont)*2/N_cont
Y_cont_log = 20*np.log10(Y_cont_norm)

# Plot the log-scale power spectra
fig = plt.figure(figsize=(10,4))
axes = fig.add_subplot(1, 1, 1)
axes.plot(np.arange(0, N_cont//2+1)*fs_cont/N_cont, Y_cont_log[0:N_cont//2+1])
axes.set_title('Power Spectra of the Continuous Signal')
axes.set_xlabel('Frequency (Hz)')
axes.set_ylabel('Power Spectra (dB)')
axes.grid(True, 'Major')

We will sample the 'continuous' signal using a sampling frequency of 200Hz. According to Nyquist Sampling Theorem, all frequencies above half the sampling rate will alias. The sampled signal is plotted below in the time domain for inspection.

In [ ]:
# Sampled signal
fs = 200
x = np.arange(0, t, 1/fs)   # Discrete time i.e. sampled time period
N = np.size(x)
y = y_cont[0::fs_cont//fs]  # Get sampled signal

# Plot the sampled signal
hf.plot_timeseries("Sampled Signal",
                   [x_cont, x], [y_cont, y], 
                   ['continuous','discrete'])

The log-scale power spectra of the sampled signal can be inspected below. Notice that the 50Hz frequency component is present in the output plot. However, there is also a 40Hz frequency component that was not in the original 'continuous' signal. This spurious frequency has emerged due to undersampling the 240Hz frequency component in the 'continuous' signal, which has caused it to alias into the sampled spectra.

In [ ]:
# Compute the sampled signal's log-scale power spectra
Y = np.fft.fft(y)
Y_norm = np.abs(Y)*2/N
Y_log = 20*np.log10(Y_norm)

# Plot the log-scale power spectra of the sampled signal
fig = plt.figure(figsize=(10,4))
axes = fig.add_subplot(1, 1, 1)
axes.plot(np.arange(0, N//2+1)*fs/N, Y_log[0:N//2+1])
axes.set_title('Power Spectra of the Sampled Signal')
axes.set_xlabel('Frequency (Hz)')
axes.set_ylabel('Power Spectra (dB)')
axes.grid(True, 'Major')

The frequency of the aliased signal can be easily computed as shown in Figure 2. The sampled signal will fold across several Nyquist Zones before residing in Nyquist Zone 1. Once you have completed this notebook, you can try different frequencies as shown in the plot i.e. 180Hz and 310Hz.

<figure>
<img src="./images/nyquist_zones_examples.png" style="width: 75%;"/>
    <figcaption><b>Figure 2: Examples of aliasing with reference to Nyquist Zones.</b></figcaption>
</figure>

### 2.2. Anti-Aliasing Filter <a class="anchor" id="anti-aliasing-filter"></a>
An anti-alias filter is used before the 'continuous' signal is sampled by the ADC. The purpose of the filter is to suppress frequencies above the Nyquist Sampling rate. We can generate a lowpass filter for this purpose below.

In [ ]:
# A simple function to generate a lowpass filter
def generate_lowpass(fs, cutoff, stop, numtaps):
    taps = signal.remez(numtaps, [0, cutoff, stop, 0.5*fs], [1, 0], Hz=fs)
    w, h = signal.freqz(taps, [1], worN=2000)
    return w, h, taps

Now use the lowpass function to create a 151 tap filter and plot the response.

In [ ]:
# Obtain the anti-alias response and plot
numtaps = 151
w, h, coeffs = generate_lowpass(fs_cont, fs/2, 1.4*(fs/2), numtaps)
hf.plot_response(fs_cont, w, h, title = "Low-Pass Filter")

The lowpass filter is convolved with the 'continuous' signal in our simulation.

In [ ]:
# Filter the 'continuous' signal
y_cont_anti = signal.filtfilt(coeffs, 1, y_cont)

Now plot the filtered 'continuous' signal for inspection.

In [ ]:
# Plot the 'continuous' signal filtered by the anti-alias filter
hf.plot_timeseries("Continuous Signal Filtered by Anti-Alias Filter",
                   [x_cont], [y_cont_anti],
                   ['continuous'])

Notice that a signal in the time domain is a 50Hz sine wave. We can plot the log-scale power spectra of the signal to verify that the 240Hz frequency component has been suppressed.

In [ ]:
# Compute the log-scale power spectra of the anti-aliased signal
Y_cont_anti = np.fft.fft(y_cont_anti)
Y_cont_anti_norm = np.abs(Y_cont_anti)*2/N_cont
Y_cont_anti_log = 20*np.log10(Y_cont_anti_norm)

# Plot the log-scale power spectra of the anti-aliased signal
fig = plt.figure(figsize=(10,4))
axes = fig.add_subplot(1, 1, 1)
axes.plot(np.arange(0, N_cont//2+1)*fs_cont/N_cont, Y_cont_anti_log[0:N_cont//2+1])
axes.set_title('Power Spectra of the Continuous Signal Filtered by the Anti-Alias Filter')
axes.set_xlabel('Frequency (Hz)')
axes.set_ylabel('Power Spectra (dB)')
axes.grid(True, 'Major')

It appears that the 240Hz sine wave is no longer present in the filtered 'continuous' signal. We can now sample the signal at 200Hz and plot the resulting log-scale power spectra below.

In [ ]:
# Sample the continuous signal filtered by the anti-alias filter
y_anti = y_cont_anti[0::fs_cont//fs]

# Compute the log-scale power spectra of the sampled filtered signal
Y_anti = np.fft.fft(y_anti)
Y_anti_norm = np.abs(Y_anti)*2/N
Y_anti_log = 20*np.log10(Y_anti_norm)

# Plot the log-scale power spectra of the sampled filtered signal
fig = plt.figure(figsize=(10,4))
axes = fig.add_subplot(1, 1, 1)
axes.plot(np.arange(0, N//2+1)*fs/N, Y_anti_log[0:N//2+1])
axes.set_title('Power Spectra of the Sampled Wave Filtered by the Anti-Alias Filter')
axes.set_xlabel('Frequency (Hz)')
axes.set_ylabel('Power Spectra (dB)')
axes.grid(True, 'Major')

The 40Hz component seen previously is no longer present. The anti-aliasing filter has successfully successfully suppressed the 240Hz sine wave and prevented aliasing. Note that this example is for suppressing frequencies outside Nyquist Zone 1. When acquiring signals in higher order Nyquist Zones, bandpass filters should be used instead to suppress the effects of aliasing.

## 3. Digital to Analogue Conversion <a class="anchor" id="dac"></a>
Reconstruction is performed when converting a signal from the digital domain into analogue. A digital to analogue converter (DAC) will hold the voltage value for a period of $ t_s$ for each of the samples giving a "steppy" signal. An example of this process is illustrated in Figure 3.

<figure>
<img src="./images/DAC_reconstruction.png" style="width: 75%;"/>
    <figcaption><b>Figure 3: DAC time domain signals and reconstruction filtering.</b></figcaption>
</figure>


### 3.1. Zero-Order-Hold
To model the signal reconstruction performed by a DAC, we can apply a zero order hold (ZOH) to the digital signal. The function below acheives this, given an input signal and a sampling frequency. 

The zero order hold function below first makes the signal "continuous" by zero padding the array to a rate of 48000 Hz - the rate we use to approximate an analogue signal in this notebook. This padded signal is then convolved with the impulse response of the ZOH, which is a rectangular window of length, $ t_s $. 

In [ ]:
def zero_order_hold(signal, fs):
    # New time axis for 'continuous' signal
    x_zoh = np.arange(0, len(signal)/fs, 1/48000)
    
    # Zero pad the original signal to new rate
    y_zoh = np.zeros(len(x_zoh))
    y_zoh_indices = y_zoh[::int(np.round(48000/fs))]
    y_zoh_indices[0:len(signal)] = signal
    y_zoh[::int(np.round(48000/fs))] = y_zoh_indices
    
    # Create ZOH impulse response and convolve with input signal
    h_zoh = np.ones(int(48000/fs))
    y_zoh = np.convolve(h_zoh,y_zoh)
    
    # Chop of end to match time axis length
    y_zoh = y_zoh[0:len(x_zoh)] 
    
    return x_zoh, y_zoh

Try changing the sampling rate to see the different outputs from the ZOH.

In [ ]:
A = 1       # Amplitude
f = 100     # Desired frequency 
t = 0.2    # Time
fs = 80    # Sampling rate

# Define 'continuous' signal for comparison
x_cont = np.arange(0, t, 1/48000)            
y_cont = A * np.sin(2 * np.pi * f * x_cont + 10)  

# Sampled signal
x = np.arange(0, t, 1/fs)                # Discrete time i.e. sampled time period
y = A * np.sin(2 * np.pi * f * x + 10)   # Formula for sine wave

hf.plot_timeseries("Sampled Signal",
                   [x_cont, x], [y_cont, y], 
                   ['continuous','discrete'])

# Perform ZOH on sampled signal
[x_zoh, y_zoh] = zero_order_hold(y, fs)
ax = hf.plot_timeseries("Zero Order Hold", 
                        [x_zoh], [y_zoh], 
                        ['continuous'])

### 3.2. Reconstruction Filter <a class="anchor" id="reconstruction-filter"></a>

Note that the output above is a little “steppy”, which is caused by the zero order hold (step reconstruction). This artefact can be removed with a reconstruction filter, which is implemented after the DAC using analogue circuitry. The reconstruction filter removes the baseband image and high frequency components present in the signal (in the form of the steps between the discrete levels).

Ideally reconstruction filters have very sharp cut-off filters at $f_{s}/2$. Steeper roll-offs are more expensive, but clearly for many applications, good analogue filters are essential. In a DSP system, the precisely trimmed analogue filters could actually be more costly than the other DSP components, and their accuracy can be affected by temperature, ageing, etc. Therefore, it is desirable to do as much of the filtering as possible in the digital domain.

To this end, we will first interpolate a sampled signal digitally, before passing it into our "analogue" reconstruction filter. Interpolation is simply upsampling a signal followed by a lowpass anti-imaging filter. Let us first design the interpolation filter below.

In [ ]:
R = 3
fs_cont = 48000
Fs = fs*R
w, h, coeffs = generate_lowpass(Fs, fs/2, 1.4*(fs/2), 15)
hf.plot_response(Fs, w, h, title = "Low-Pass Filter")

Now apply the interpolation filter to the discrete signal.

In [ ]:
x_ov = np.arange(0, t, 1/(Fs))
y_ov = np.zeros(len(y)*R)
y_ov[::R] = y

y_lp = signal.filtfilt(coeffs, 1, y_ov) * R
hf.plot_timeseries("Interpolated Signal, R = 3",
                   [x_ov], [y_lp],
                  ['discrete'])

A Zero Order Hold is then applied to the interpolated signal to model the DAC sampling process. The result is plotted below for inspection.

In [ ]:
# Perform ZOH on sampled signal
[x_zoh, y_zoh] = zero_order_hold(y_lp, Fs)
hf.plot_timeseries("Zero-Order-Hold",
                   [x_zoh], [y_zoh],
                   ['continuous'])

A reconstruction filter with 400 filter taps is designed, which simulates an analogue reconstruction filter that is situated after the DAC. The filter response can be visualised below. The passband is very small, only allowing a few frequencies to remain and the rest are suppressed.

In [ ]:
w, h, coeffs = generate_lowpass(fs_cont, fs/2, Fs/2, 400)
hf.plot_response(fs_cont, w, h, title = "Low-Pass Filter")

The output of the reconstruction filter can be plotted alongside the step response for comparison. This plot is generated running the code cell below. Notice that the output of the filter is smooth and does not contain abrupt changes in amplitude.

In [ ]:
# [x_zoh, y_zoh] = zero_order_hold(y, fs)
y_recon = signal.filtfilt(coeffs, 1, y_zoh)
hf.plot_timeseries("Reconstructed Signal and Zero-Order-Hold Signal",
                   [x_zoh, x_zoh], [y_zoh, y_recon],
                   ['continuous','continuous'])

The smooth output response is more suitable for transmission as it does not contain spurious high frequency components that are present in the steppy output signal of the DAC.

## 4. Conclusion <a class="anchor" id="conclusion"></a>
This notebook has simulated the analogue to digital conversion process and explored anti-aliasing filters. Additionally, the digital to analogue conversion process was investigated and reconstruction filters were discussed.

In the next notebook, we review digital filters and design techniques.

---

[⬅️ Previous Notebook](02_quantisation.ipynb) || [Next Notebook 🚀](04_digital_filter_design.ipynb)

Copyright © 2023 Strathclyde Academic Media

---
---